# Kimore

In [1]:
import os
import csv
import openpyxl
import pandas as pd
import numpy as np

path = "Kimore"
kinect_joints = ["spinebase", "spinemid", "neck", "head", 
                 "shoulderleft", "elbowleft", "wristleft", 
                 "handleft", "shoulderright", "elbowright", 
                 "wristright", "handright", "hipleft", "kneeleft", 
                 "ankleleft", "footleft", "hipright", "kneeright", 
                 "ankleright", "footright", "spineshoulder", "handtipleft", 
                 "thumbleft", "handtipright", "thumbright"]

enable_kinect_joints = False

data = []
for (root, dirs, files) in os.walk(path):

    # if current directory contains "Raw", extract data
    if "Raw" in dirs:
        
        new_dict = {}
        
        # get exercise number
        new_dict["Exercise"] = int(root[-1])

        # extract raw data
        raw_files = os.listdir(os.path.join(root, "Raw"))
        for file in raw_files:

            file_path = os.path.join(os.path.join(root, "Raw"),file)
            csv_file = open(file_path, newline='')
            csv_reader = csv.reader(csv_file)
            
            if file.startswith("JointOrientation"):
                
                if enable_kinect_joints:
                    for joint in kinect_joints:
                        new_dict[joint + "-o"] = []

                    for row in csv_reader:
                        for i in range(len(kinect_joints)):
                            if len(row) > 0:
                                new_dict[kinect_joints[i] + "-o"].append(row[(4*i):(4*i+4)])
                else:
                    new_dict["JointOrientation"] = []
                    for row in csv_reader:
                        if len(row) > 0:
                            new_dict["JointOrientation"].append(row)

            elif file.startswith("JointPosition"):
                
                if enable_kinect_joints:
                    for joint in kinect_joints:
                        new_dict[joint + "-p"] = []

                    for row in csv_reader:
                        for i in range(len(kinect_joints)):
                            if len(row) > 0:
                                new_dict[kinect_joints[i] + "-p"].append(row[(4*i):(4*i+3)])
                else:
                    new_dict["JointPosition"] = []
                    for row in csv_reader:
                        if len(row) > 0:
                            new_dict["JointPosition"].append(row)

            elif file.startswith("TimeStamp"):

                new_dict["Timestamps"] = []
                for row in csv_reader:
                    if len(row) > 0:
                        new_dict["Timestamps"].append(row[0])

        # extract data labels
        label_files = os.listdir(os.path.join(root, "Label"))
        for file in label_files:

            file_path = os.path.join(os.path.join(root, "Label"),file)
            book = openpyxl.load_workbook(file_path)
            sheet = book.active

            if file.startswith("SuppInfo"):
                for i in range(1, sheet.max_column):
                    t = sheet.cell(1, i).value
                    v = sheet.cell(2, i).value
                    new_dict[t] = v
            
            elif file.startswith("ClinicalAssessment"):
                new_dict["cTS"] = sheet.cell(2, new_dict["Exercise"]+1).value
                new_dict["cPO"] = sheet.cell(2, new_dict["Exercise"]+6).value
                new_dict["cCF"] = sheet.cell(2, new_dict["Exercise"]+11).value
        
        # append exercise to data
        data.append(new_dict)

C:\Users\Dilemre\anaconda3\lib\site-packages\openpyxl\reader\excel.py:228: UserWarning: Unknown extension is not supported and will be removed
  ws_parser.bind_all()


In [28]:
df = pd.DataFrame(data)

In [29]:
df

,Exercise,JointOrientation,JointPosition,Timestamps,cTS,cPO,cCF,Subject ID,Group,Age
0,1,"[[-0.0531085, 0.00232358, 0.99423, -0.0931733,...","[[-0.0468451, -0.409343, 2.6241, 2, -0.0484949...","[20622112895, 20622442942, 20622783803, 206231...",48.333333,14.666667,33.666667,E_ID1,E,50
1,2,"[[-0.0610258, 0.00258101, 0.995022, -0.0787486...","[[-0.0578279, -0.407365, 2.60017, 2, -0.059207...","[21184779662, 21185108840, 21185438830, 211857...",45.000000,13.666667,31.333333,E_ID1,E,50
2,3,"[[-0.0298744, -0.00814489, 0.995711, -0.087179...","[[-0.0652774, -0.418544, 2.56966, 2, -0.071836...","[21902433786, 21902774551, 21903104529, 219034...",48.333333,15.000000,33.333333,E_ID1,CG_E,50
3,4,"[[-0.0369213, -0.00490366, 0.995614, -0.085821...","[[-0.0674865, -0.397115, 2.58841, 2, -0.072328...","[22621099365, 22621428603, 22621769446, 226220...",50.000000,15.000000,35.000000,E_ID1,CG_E,50
4,5,"[[-0.0517414, -0.000435555, 0.99603, -0.072440...","[[-0.0630149, -0.394823, 2.59342, 2, -0.065548...","[23248424127, 23248764907, 23249094898, 232494...",48.333333,15.000000,33.333333,E_ID1,CG_E,50
...,...,...,...,...,...,...,...,...,...,...
385,1,"[[0.039838, -0.0277761, 0.995785, -0.0777996, ...","[[-0.20825, -0.500367, 2.32432, 2, -0.223546, ...","[711302136484, 711302477267, 711302806451, 711...",26.333333,6.333333,20.000000,S_ID9,S,67
386,2,"[[0.0600943, -0.0332151, 0.995777, -0.0609322,...","[[-0.209425, -0.48988, 2.31622, 2, -0.227487, ...","[711894802395, 711895132382, 711895473180, 711...",17.666667,4.333333,13.333333,S_ID9,S,67
387,3,"[[0.0795781, -0.0310581, 0.994218, -0.0650627,...","[[-0.202571, -0.492478, 2.31694, 2, -0.218464,...","[712329470249, 712329799436, 712330129378, 712...",26.333333,6.333333,20.000000,S_ID9,S,67
388,4,"[[-0.00995722, -0.0160432, 0.997548, -0.067393...","[[-0.199238, -0.471849, 2.29448, 2, -0.209099,...","[712712796766, 712713126759, 712713467555, 712...",23.000000,7.000000,16.000000,S_ID9,S,67


In [30]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Exercise,390.0,3.000000,1.416030,1.000000,2.000000,3.000000,4.0,5.0
cTS,383.0,37.457573,10.204780,10.000000,30.140452,40.000000,46.0,50.0
cPO,383.0,11.040233,3.952683,0.005147,8.540081,12.000000,14.5,15.0
cCF,383.0,26.413858,7.043139,7.000000,22.276553,27.360633,32.0,35.0
Age,390.0,45.641026,19.354609,17.000000,26.000000,45.500000,63.0,81.0


In [31]:
df.isnull().sum()

Exercise            0
JointOrientation    5
JointPosition       5
Timestamps          3
cTS                 7
cPO                 7
cCF                 7
Subject ID          0
Group               0
Age                 0
dtype: int64

In [32]:
dropped_rows = df[df.isna().any(axis=1)]
dropped_rows

,Exercise,JointOrientation,JointPosition,Timestamps,cTS,cPO,cCF,Subject ID,Group,Age
40,1,"[[-0.0481282, 8.87374e-007, 0.985768, 0.161073...","[[0.175871, -1.10702, 2.84095, 2, 0.180718, -0...","[30437599246420, 30437599576380, 3043759990709...",NaN,NaN,NaN,E_ID17,E,45
41,2,"[[0.0199295, -0.0368523, 0.996205, 0.0762919, ...","[[0.155424, -1.09849, 3.29907, 2, 0.13016, -0....","[30438448240198, 30438448570198, 3043844890094...",NaN,NaN,NaN,E_ID17,E,45
42,3,"[[-0.228232, 0.030343, 0.964276, 0.131002, -0....","[[0.173645, -1.03028, 3.27985, 2, 0.209115, -0...","[30439025566426, 30439025896688, 3043902623600...",NaN,NaN,NaN,E_ID17,E,45
43,4,"[[-0.101832, 0.0088296, 0.98639, 0.128791, -0....","[[0.277114, -1.03271, 3.01552, 2, 0.290556, -0...","[30442081214560, 30442081543751, 3044208188454...",NaN,NaN,NaN,E_ID17,E,45
44,5,"[[-0.0201885, -0.0603588, 0.992337, -0.105904,...","[[0.300576, -1.3918, 3.14916, 2, 0.263258, -1....","[30442714879944, 30442715209965, 3044271553914...",NaN,NaN,NaN,E_ID17,E,45
76,2,NaN,NaN,NaN,NaN,NaN,NaN,E_ID8,E,22
97,3,NaN,"[[-0.196504, -0.761454, 2.62517, 2, -0.197067,...","[10369220941503, 10369221270654, 1036922160065...",40.00000,15.000000,25.000000,NE_ID11,NE,75
99,5,[],NaN,"[10370694259997, 10370694589964, 1037069493075...",49.00000,14.000000,35.000000,NE_ID11,NE,75
166,2,"[[0.0367658, 0.00355401, 0.998187, -0.0475269,...",NaN,"[184099250059, 184099580040, 184099910803, 184...",44.00000,13.000000,31.000000,NE_ID24,NE,25
258,4,NaN,NaN,NaN,NaN,NaN,NaN,B_ID8,B,36


In [33]:
df = df.dropna().reset_index(drop=True)

In [34]:
for i in range(0, len(df["JointPosition"])):
    for j in range(0, len(df["JointPosition"][i])):
        if len(df["JointPosition"][i][j]) != 101:
            print("Position:\t", i, j, "len: ",len(df["JointPosition"][i][j]))
            print("Orientation:\t", i, j, "len: ",len(df["JointOrientation"][i][j]))
            break

Position:	 70 291 len:  201
Orientation:	 70 291 len:  201
Position:	 305 992 len:  201
Orientation:	 305 992 len:  201
Position:	 362 17 len:  201
Orientation:	 362 17 len:  201


In [35]:
signal = 1
for i in range(0, len(df["JointPosition"])):
    for j in range(0, len(df["JointPosition"][i])):
        if np.shape(df["JointPosition"]) != np.shape(df["JointOrientation"]):
            print("Position:\t", i, j, "len: ",len(df["JointPosition"][i][j]))
            print("Orientation:\t", i, j, "len: ",len(df["JointOrientation"][i][j]))
            signal = 0
    
if signal:
    print("All equally shaped")

All equally shaped


In [41]:
group = df.Group.unique()
exercise = df.Exercise.unique()

In [65]:
for g in group:
    for e in exercise:
        print("Group:", g, "\tExercise:", e, "\tCount:", df[(df["Group"]==g) & (df["Exercise"]==e)].shape[0])

Group: E 	Exercise: 1 	Count: 16
Group: E 	Exercise: 2 	Count: 15
Group: E 	Exercise: 3 	Count: 15
Group: E 	Exercise: 4 	Count: 15
Group: E 	Exercise: 5 	Count: 15
Group: CG_E 	Exercise: 1 	Count: 0
Group: CG_E 	Exercise: 2 	Count: 0
Group: CG_E 	Exercise: 3 	Count: 1
Group: CG_E 	Exercise: 4 	Count: 1
Group: CG_E 	Exercise: 5 	Count: 1
Group: NE 	Exercise: 1 	Count: 27
Group: NE 	Exercise: 2 	Count: 26
Group: NE 	Exercise: 3 	Count: 26
Group: NE 	Exercise: 4 	Count: 27
Group: NE 	Exercise: 5 	Count: 26
Group: B 	Exercise: 1 	Count: 8
Group: B 	Exercise: 2 	Count: 8
Group: B 	Exercise: 3 	Count: 8
Group: B 	Exercise: 4 	Count: 7
Group: B 	Exercise: 5 	Count: 8
Group: P 	Exercise: 1 	Count: 16
Group: P 	Exercise: 2 	Count: 16
Group: P 	Exercise: 3 	Count: 16
Group: P 	Exercise: 4 	Count: 16
Group: P 	Exercise: 5 	Count: 15
Group: S 	Exercise: 1 	Count: 10
Group: S 	Exercise: 2 	Count: 10
Group: S 	Exercise: 3 	Count: 9
Group: S 	Exercise: 4 	Count: 10
Group: S 	Exercise: 5 	Count: 10
